In [56]:
import baltic as bt
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import timedelta
import time
#import pymc3
import math
import arviz as az
#from hpd import hpd
import scipy.stats as stats
from io import StringIO
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
import seaborn as sns

from zipfile import ZipFile
import scipy as sp


import sys, subprocess, glob, os, shutil, re, importlib
from subprocess import call
import imp


%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import matplotlib.colors as clr
from matplotlib import rc
import textwrap as textwrap
from textwrap import wrap


from scipy.special import binom

In [57]:
log_file_path = "/Users/mparedes/Desktop/mpox_rhino/monthly_pred/550_mascot_skyline_one_march.log"


In [58]:
burnin_percent = 0.30

In [59]:
def read_in_Ne_changes_mascot(log_file_path):
    
    Ne_skyline_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                #print(line)
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    Ne_column_indices = []   # list to store column indices
                    Nes_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "Ne." in col:
                            Ne_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in Ne_column_indices:
                        name = line.split("\t")[n]
                        deme = name.split(".")[1]# the syntax here is "Ne.region.1" where region is deme and 1 is interval 1
                        interval = name.split(".")[2]
                       
                        Nes_key[n] = name
                        Ne_skyline_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    Ne_skyline_dict["sample"].append(sample)

                    for index in Ne_column_indices:
                        name = Nes_key[index]
                        Ne_skyline_dict[name].append(line.split("\t")[index])
                    
                
    return(Ne_skyline_dict)

In [60]:
Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

In [61]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "SkylineNe" in i:
            deme = i.split(".")[1]
            #print(deme)
            interval = i.split(".")[2]
            #print(interval)
            #print(interval)
            #print(i)
            next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = 10**mean_log
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)
            
            try:
                next_local_series = input_df["SkylineNe"+"."+ str(deme) +"." + str(next_interval)].astype('float').to_numpy()
                diff_series = np.subtract(local_series, next_local_series)
                #print(local_series)
                #print(next_local_series)
                #print(diff_series)
                diff_mean_log = diff_series.mean()
                diff_median_log = np.median(diff_series)
                diff_hpd_95 = az.hdi(diff_series, 0.95)
                diff_lower_hpd_log_95 = diff_hpd_95[0]
                diff_lower_hpd_linear_95 = math.exp(diff_lower_hpd_log_95)
                diff_upper_hpd_log_95 = diff_hpd_95[1]
                diff_upper_hpd_linear_95 = math.exp(diff_upper_hpd_log_95)
                diff_hpd_50 = az.hdi(diff_series, 0.50)
                diff_lower_hpd_log_50 = diff_hpd_50[0]
                diff_lower_hpd_linear_50 = math.exp(diff_lower_hpd_log_50)
                diff_upper_hpd_log_50 = diff_hpd_50[1]
                diff_upper_hpd_linear_50 = math.exp(diff_upper_hpd_log_50)
            except KeyError:
                pass   
            
            try:
                local_df = pd.DataFrame.from_dict({"deme":deme, "interval":interval, "mean_Ne_log":mean_log,"mean_Ne_linear":mean_linear, 
                                                   "median_Ne_log" : median_log, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                   "diff_mean_Ne_log":diff_mean_log, "diff_mean_log":diff_mean_log,
                                                   "diff_upper_hpd_log_95":diff_upper_hpd_log_95,"diff_lower_hpd_log_95":diff_lower_hpd_log_95, 
                                                   "diff_upper_hpd_log_50":diff_upper_hpd_log_50,"diff_lower_hpd_log_50":diff_lower_hpd_log_50,
                                                   "diff_upper_hpd_linear":diff_upper_hpd_linear_95,"diff_lower_hpd_linear":diff_lower_hpd_linear_95,
                                                   "diff_upper_hpd_linear_50":diff_upper_hpd_linear_50,"diff_lower_hpd_linear_50":diff_lower_hpd_linear_50})
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
    return(new_df)

In [62]:
Ne_df = pd.DataFrame.from_dict(Ne_skyline)
print(len(Ne_df))
Ne_df

8391


,sample,SkylineNe.CentralEurope.1,SkylineNe.CentralEurope.2,SkylineNe.CentralEurope.3,SkylineNe.CentralEurope.4,SkylineNe.CentralEurope.5,SkylineNe.CentralEurope.6,SkylineNe.CentralEurope.7,SkylineNe.CentralEurope.8,SkylineNe.CentralEurope.9,...,SkylineNe.WesternEurope.2,SkylineNe.WesternEurope.3,SkylineNe.WesternEurope.4,SkylineNe.WesternEurope.5,SkylineNe.WesternEurope.6,SkylineNe.WesternEurope.7,SkylineNe.WesternEurope.8,SkylineNe.WesternEurope.9,SkylineNe.WesternEurope.10,SkylineNe.WesternEurope.11\n
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0\n
1,5000,-1.0858811746983144,0.1133093584343813,4.066649423074729,1.361350686677951,0.41739974801661506,-0.716382376991204,0.2397262028295704,1.4249125141572243,-0.6327925035594484,...,-2.266732789627967,-1.062491041545816,0.37755240686904035,-0.32207969190734326,-1.2910334514931363,-1.1557888339979283,0.16226001031859127,-0.20210336235037474,-2.68280366040835,-0.5924148888990907\n
2,10000,-0.5774021109852073,0.19076304771968963,4.469931694586019,1.2850476316250599,-1.213459203510489,-1.7502463847296763,-0.06786970339404713,0.08869941662206621,-0.05312057854063354,...,-2.7682729743461656,-2.434523997105149,0.570804916446418,1.1588627731627978,-0.694933523795098,-1.5864933142473683,-0.5612961461662123,-1.264434555699605,-1.8293358768566066,-1.9681808602388984\n
3,15000,-0.16850185102536716,0.27753911682874005,4.146923779797415,1.8079009033630329,-2.4067067986573507,-2.3438043054838467,-0.4827187619124544,0.3891010184942947,1.1550606771662717,...,-0.4252754034848839,-0.25636977991926946,1.3773033226751537,0.3631090542372688,-0.7261561654521268,-0.6151323827782545,-0.05172104392546254,0.09792557082669663,-0.34681778561037035,-1.0419171213027956\n
4,20000,-0.2377788054874682,1.6553268480975933,4.2240340634242415,1.819988727828659,-1.0438583823856113,-0.9408981673101438,-0.5426337986280771,1.1467911953382788,1.743084807760368,...,-0.2644073291768322,1.348899293352832,1.990496271622251,0.16819724181724816,-0.35615678141342744,-1.0049617567363915,0.055069921676184824,-1.6220502087950415,-1.6558968241912735,-1.8698579178875634\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8386,41930000,-0.237091880672131,-0.3992900362615665,-0.7247058781156676,-1.4310403299052752,-0.7874841731251804,-1.3437140382860904,-1.7585652267890868,-2.687377314991107,-2.3122994298677693,...,1.1023962735064692,0.9842680369986428,0.3635968237267215,2.0648588688346488,3.7135199186837657,3.1747534169805154,1.4089327903995834,-1.5168813406188162,-3.048110676792508,-2.363078013334785\n
8387,41935000,0.8172383406723317,0.6974931743375411,-0.5534762043603966,-0.36620786069669936,-0.5854210842592499,-0.6289542344046122,-1.1312436506349646,-2.132070101022517,-2.9468634423319284,...,-0.27979472870873023,0.6569144983851529,1.0210523020074855,2.457767774731457,3.613463339841674,2.8578125326374226,1.356952459332548,-1.4751869347984772,-3.370101857447194,-2.4981182779878583\n
8388,41940000,1.1179502883090986,0.9463558954980683,-0.4672278426374847,-1.1552027499967288,-0.941901764111846,-0.8223948126317181,-0.7681062492412014,-2.7739727896653448,-2.115804052214968,...,-1.0797792087525235,0.5218051985965885,1.0687623543344418,2.637211543192867,3.4638093818389133,3.061942209963458,1.7566510986969586,-1.2715969215553786,-2.8183529897507236,-0.5845402388617144\n
8389,41945000,1.453210899828253,0.828849316739001,-0.26542355234846077,-0.8557352410496286,-1.2569307065227606,-0.8817377634256128,-1.126177338007263,-3.1329595406400848,-3.3804544764193794,...,-1.4715799174560327,0.4080093293778121,0.7987803728850306,2.805158372026043,3.358075886844034,3.0355032965286513,1.6579786834926236,-1.6896520780238018,-2.4376112479008425,-0.6516330108129578\n


In [63]:
ne_summary = generate_summary_df(Ne_df)
ne_summary

/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_76406/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_76406/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_76406/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_76406/308575421.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)


,deme,interval,mean_Ne_log,mean_Ne_linear,median_Ne_log,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,upper_hpd_linear,...,diff_mean_Ne_log,diff_mean_log,diff_upper_hpd_log_95,diff_lower_hpd_log_95,diff_upper_hpd_log_50,diff_lower_hpd_log_50,diff_upper_hpd_linear,diff_lower_hpd_linear,diff_upper_hpd_linear_50,diff_lower_hpd_linear_50
0,CentralEurope,1,-0.058043,0.874897,-0.065036,1.744639,-1.855207,0.686485,-0.578673,5.723834,...,0.115257,0.115257,1.819007,-1.708411,0.811451,-0.431612,6.165730,0.181153,2.251171,0.649462
0,CentralEurope,2,-0.173300,0.670965,-0.171907,2.070105,-2.429845,0.579249,-0.959313,7.925655,...,0.111775,0.111775,1.907330,-1.588808,0.706894,-0.464393,6.735084,0.204169,2.027684,0.628517
0,CentralEurope,3,-0.285075,0.518710,-0.287587,1.888653,-2.688074,0.451467,-1.089830,6.610457,...,0.033795,0.033795,1.847218,-1.804728,0.564671,-0.672891,6.342150,0.164519,1.758868,0.510231
0,CentralEurope,4,-0.318870,0.479877,-0.338168,1.739397,-2.513301,0.357711,-0.975782,5.693910,...,-0.146091,-0.146091,1.668705,-2.132410,0.331573,-0.898808,5.305292,0.118551,1.393158,0.407054
0,CentralEurope,5,-0.172779,0.671771,-0.157089,1.386191,-1.762308,0.405035,-0.631267,3.999587,...,0.359935,0.359935,2.345261,-1.495077,0.908601,-0.393859,10.435996,0.224231,2.480850,0.674449
0,CentralEurope,6,-0.532714,0.293282,-0.442981,1.061385,-2.325598,0.113610,-0.878916,2.890370,...,1.436588,1.436588,3.152686,-0.295505,1.910393,0.734693,23.398826,0.744156,6.755743,2.084842
0,CentralEurope,7,-1.969303,0.010732,-1.897521,-0.105068,-4.022934,-1.050594,-2.398178,0.900264,...,1.109504,1.109504,3.034765,-0.929321,2.023207,0.605519,20.796092,0.394822,7.562538,1.832203
0,CentralEurope,8,-3.078807,0.000834,-3.045187,-1.051460,-4.987142,-2.384419,-3.634150,0.349427,...,0.382856,0.382856,2.318608,-1.574454,1.003453,-0.322674,10.161519,0.207121,2.727684,0.724210
0,CentralEurope,9,-3.461662,0.000345,-3.462592,-0.974923,-5.992458,-2.534076,-4.172990,0.377221,...,0.144781,0.144781,2.103644,-1.777285,0.830534,-0.482817,8.195986,0.169097,2.294543,0.617043
0,CentralEurope,10,-3.606443,0.000247,-3.560247,-0.713233,-6.804745,-2.478705,-4.473108,0.490057,...,0.005945,0.005945,1.968427,-1.948806,0.716488,-0.640682,7.159404,0.142444,2.047231,0.526933


In [64]:
ne_summary.lower_hpd_linear_50

0     0.560642
0     0.383156
0     0.336274
0     0.376897
0     0.531918
0     0.415233
0     0.090883
0     0.026406
0     0.015406
0     0.011412
0     0.006985
0     0.938963
0     1.524048
0     1.776031
0     3.790894
0     4.979626
0     2.877965
0     1.253130
0     0.358987
0     0.047754
0     0.015904
0     0.013728
0     0.793000
0     1.139406
0     2.270291
0     2.274263
0     0.404182
0     0.105270
0     0.035473
0     0.016802
0     0.010826
0     0.008419
0     0.006846
0     0.580528
0     0.515203
0     0.615189
0     0.989763
0     2.035780
0     2.458625
0     1.665119
0     0.089433
0     0.036325
0     0.026572
0     0.015403
0     0.608524
0     0.722363
0     0.817634
0     2.203494
0    11.135331
0    14.011106
0     6.469000
0     0.261286
0     0.057575
0     0.024650
0     0.019622
Name: lower_hpd_linear_50, dtype: float64

In [65]:
test = ne_summary
test['days'] = (test.interval.astype(int))*25
test['date'] = dt.strptime("2023-01-03",  "%Y-%m-%d") - test.days.map(timedelta)
#test.date = test.date.astype(str)

In [66]:
domain = ['CentralEurope', 'NorthAmerica', 'SouthAmerica' ,"SouthernEurope", "WesternEurope"]
range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]
line = alt.Chart(test).mark_area(interpolate='monotone', opacity = 0.5).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False, format="%B %Y")),
    alt.Y('lower_hpd_linear_50',axis=alt.Axis(title="Effective Population Size", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_linear_50' ),
    color=alt.Color('deme:N', scale=alt.Scale(domain=domain, range=range_) )
).properties(
    width=850,
    height=400
).transform_filter(
    (datum.upper_hpd_linear < 60) &(datum.upper_hpd_linear_50 < 10)
)

# band = alt.Chart(test).mark_area(
#     opacity=0.3, interpolate='monotone'
# ).encode(
#     alt.X('date:T', axis=alt.Axis(title="", grid=False)),
#     alt.Y('lower_hpd_linear'),#axis=None),#, scale=alt.Scale(domain=(0, 13))),
#     alt.Y2('upper_hpd_linear'),
#     color=alt.Color('deme:N', title = "Region", legend=alt.Legend(orient = "right",offset = -135, labelFontSize = 16, titleFontSize = 16, symbolSize = 110))
# ).properties(
#     width=850,
#     height=400
# ).transform_filter(
#     (datum.upper_hpd_linear < 20) &(datum.upper_hpd_linear_50 < 30)
# )
# band
# ne_plot = line + band
# (ne_plot).configure_axis(
#     labelFontSize=20,
#     titleFontSize=20
# )
line

alt.Chart(...)